<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/model_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

emmanueleyramagbetor_dataset_eczema_path = kagglehub.dataset_download('emmanueleyramagbetor/dataset-eczema')

print('Data source import complete.')


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets scikit-learn peft transformers


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-3hn9us80/unsloth_de77b9d3ae1c4122bd3bf635ef5ae1b5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-3hn9us80/unsloth_de77b9d3ae1c4122bd3bf635ef5ae1b5
  Resolved https://github.com/unslothai/unsloth.git to commit 872e191494104d177d854a2e2b430a03a193d4e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 18.6 M

In [ ]:
from datasets import load_dataset

data_files = {
    "test": "/kaggle/input/dataset-eczema/test.csv",
}

dataset = load_dataset("csv", data_files=data_files)
test_ds = dataset["test"]
test_ds


Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [ ]:
def format_for_eval(example):
    prompt = f"Description: {example['text']}\n\nAnswer the correct class (0 or 1)."
    example["prompt"] = prompt
    return example

test_ds = test_ds.map(format_for_eval)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel

base_id = "google/medgemma-4b-it"

base_model, base_processor = FastLanguageModel.from_pretrained(
    model_name = base_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
base_processor.tokenizer.padding_side = "right"


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-25 00:44:37.504121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764031477.691277      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764031477.743108      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.12G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
import torch

def predict(model, processor, text):
    inputs = processor(text=[text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated = model.generate(**inputs, max_new_tokens=5)
    out = processor.tokenizer.decode(generated[0], skip_special_tokens=True)

    # Extract final digit (very reliable for binary)
    if "1" in out[-3:]:
        return 1
    if "0" in out[-3:]:
        return 0
    return 0


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model(model, processor, ds):
    preds, labels = [], []

    for ex in ds:
        pred = predict(model, processor, ex["prompt"])
        preds.append(pred)
        labels.append(int(ex["label"]))

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
print("Evaluating BASE medgemma-4b-it...")
base_scores = evaluate_model(base_model, base_processor, test_ds)
base_scores


Evaluating BASE medgemma-4b-it...


{'accuracy': 0.655,
 'precision': 0.9112903225806451,
 'recall': 0.4708333333333333,
 'f1': 0.6208791208791208}

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from peft import PeftModel

lora_model = PeftModel.from_pretrained(
    base_model,
    "aee4/medgemma-eczema",
)
lora_model.eval()


adapter_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/154M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-15): 16 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): lora.Linear(
                      (base_layer): Linear(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lor

In [ ]:
print("Evaluating BASE + ADAPTER (finetuned)...")
lora_scores = evaluate_model(lora_model, base_processor, test_ds)
lora_scores


Evaluating BASE + ADAPTER (finetuned)...


{'accuracy': 0.683,
 'precision': 0.6543075245365322,
 'recall': 1.0,
 'f1': 0.7910349373764008}